In [2]:
import os
import glob
import pickle
import numpy as np
import pandas as pd  
import csv
import string
from scipy.io import savemat
from scipy.io import loadmat
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize
from functools import reduce 
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
import argparse
import torch
from torch import nn
from einops import rearrange
import torch.utils.data as data
from torch.nn import functional as F
from torch.optim import SGD, Adam
from torch.optim.lr_scheduler import CosineAnnealingLR
from torchsummary import summary

In [3]:
batch_size=50
num_words=500
emb_dim=50
num_lags=3
EPOCHS = 5


electrode=5  #which electrode are we considering
taking_words=False  # are we taking the actual words or the lags around the last word
lag_number=3 # how many previous lags/words to consider
lags=[0,-50,50] # only valid if taking word is false and should equal to the lag_number
train_num=3500  
subject=798
min_num_words=5 

## Data Prepare

In [7]:
# embeddings=np.random.randn(num_words, 1,emb_dim)
# elec_data=np.random.randn(num_words, 1,num_lags)

# X_train=torch.from_numpy(embeddings[:400,:,:])
# y_train=torch.from_numpy(elec_data[:400,:,:])

# X_test=torch.from_numpy(embeddings[400:,:,:])
# y_test=torch.from_numpy(elec_data[400:,:,:])

# trainset = torch.utils.data.TensorDataset(X_train, y_train)
# trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, drop_last=True)

# testset = torch.utils.data.TensorDataset(X_train, y_train)
# testloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, drop_last=True)



In [8]:
def load_pickle(file_path):
    
    pickle_file = open(file_path, "rb")
    objects = []

    i=0

    while True:
        # print('i',i)
        try:

            objects.append(pickle.load(pickle_file))

        except EOFError:

            break

    pickle_file.close()

    a=objects[0]
    
    return a

def load_label(filepath):

    with open(filepath, "rb") as f:
        full_labels = pickle.load(f)
        #labels_df = pd.DataFrame(full_labels["labels"])
        labels_df = pd.DataFrame(full_labels)

    # labels_df["audio_onset"] = ((labels_df.onset + 3000) / 512)
    # labels_df["audio_offset"] = ((labels_df.offset + 3000) / 512)

    # labels_df = labels_df.dropna(subset=["audio_onset", "audio_offset"])
    

    return labels_df


In [9]:
def get_elec_id(subject):

    path='/projects/HASSON/247/plotting/sig-elecs/20230510-tfs-sig-file/'
    
    sig_file=path+'tfs-sig-file-glove-'+ str(subject)+'-'+'comp'+'.csv'
    
    df_sig=pd.read_csv(sig_file)
    
    elecs=df_sig.electrode.values
    
    
    
    path='/scratch/gpfs/kw1166/247/247-pickling/results/tfs/'+str(subject)+'/pickles/'+str(subject)+'_electrode_names.pkl'
    
    pickle_file = open(path, "rb")
    objects = []
    
    i=0
    
    while True:
        # print('i',i)
        try:
    
            objects.append(pickle.load(pickle_file))
    
        except EOFError:
    
            break
    
    pickle_file.close()
    
    a=objects[0]
    
    df_name_match=pd.DataFrame(a)
    
    elec_id=[]
    for elec in elecs:
        elec_id.extend(df_name_match[df_name_match.electrode_name==elec].electrode_id.values)
                      
    return elec_id

def get_elec_data(subject, df, conv_name, all_onsets, lags, elec_id, lag_number,taking_words=True):

    path='/projects/HASSON/247/data/conversations-car/'+str(subject)+'/'
    # conv_name=df[df.conversation_id==conv_id].conversation_name.values[0]
    # q1=glob.glob("*_comp_Y.npy")
    # q1=glob.glob("NY625_*") ## all the conversation
    # # path_elec=path+'/'+q1[0]+'/preprocessed/'
    
    elec_num=len(elec_id)
    
    # Y_data= np.zeros((len(lags),len(onsets),elec_num)) 

    Y_data= np.zeros((elec_num, lag_number)) 

    ecogs=[]
    for k1 in elec_id:
        filename=path+'/'+conv_name+'/preprocessed/'+conv_name+'_electrode_preprocess_file_'+str(k1)+'.mat'

        if subject==798:
            filename=path+'/'+conv_name+'/preprocessed_allElec/'+conv_name+'_electrode_preprocess_file_'+str(k1)+'.mat'
        
        e=loadmat(filename)['p1st'].squeeze().astype(np.float32)
        ecogs.append(e)
    
    ecogs = np.asarray(ecogs).T
    t=len(ecogs[:,0])
    
    window_size=200
    half_window = round((window_size / 1000) * 512 / 2)

    if taking_words:

        onsets=all_onsets[-lag_number:-1]
        onsets=np.append(onsets,all_onsets[-1])
    else:

        for i in lags:

            lag_amount = int(lags[i]/ 1000 * 512)

            onsets.append(np.minimum(
                t - half_window - 1,
                np.maximum(half_window + 1,
                            np.round(all_onsets[-1], 0, all_onsets[-1]) + lag_amount)))

    
    
    index_onsets=np.asarray(onsets)
    print(index_onsets)

    print(np.shape(index_onsets))
    
    # Y= np.zeros((len(onsets), np.shape(ecogs)[1]))
    
        
    # for ii in range(len(lags)):
   

    for k in range(np.shape(ecogs)[1]):

        Y1 = np.zeros((len(onsets), 2 * half_window + 1))
        brain_signal=ecogs[:,k]
        
        # subtracting 1 from starts to account for 0-indexing
        starts = index_onsets - half_window - 1
        stops = index_onsets + half_window
        
        for i, (start, stop) in enumerate(zip(starts, stops)):
            start=int(start)
            stop=int(stop)
            Y1[i, :] = brain_signal[start:stop].reshape(-1)
            
                            
        #if subject==717:
           
        Y_data[k,:] = np.mean(Y1, axis=-1)

    return Y_data


In [10]:
def get_elec_data(subject, df, ecogs, conv_name, all_onsets, lags, elec_id, lag_number,taking_words=True):

    # path='/projects/HASSON/247/data/conversations-car/'+str(subject)+'/'
    # # conv_name=df[df.conversation_id==conv_id].conversation_name.values[0]
    # # q1=glob.glob("*_comp_Y.npy")
    # # q1=glob.glob("NY625_*") ## all the conversation
    # # # path_elec=path+'/'+q1[0]+'/preprocessed/'
    
    
    
    # # Y_data= np.zeros((len(lags),len(onsets),elec_num)) 



    # ecogs=[]
    # for k1 in elec_id:
    #     filename=path+'/'+conv_name+'/preprocessed/'+conv_name+'_electrode_preprocess_file_'+str(k1)+'.mat'

    #     if subject==798:
    #         filename=path+'/'+conv_name+'/preprocessed_allElec/'+conv_name+'_electrode_preprocess_file_'+str(k1)+'.mat'
        
    #     e=loadmat(filename)['p1st'].squeeze().astype(np.float32)
    #     ecogs.append(e)
    
    # ecogs = np.asarray(ecogs).T



    elec_num=len(elec_id)
    t=len(ecogs[:,0])   
    window_size=200
    half_window = round((window_size / 1000) * 512 / 2)

    if taking_words:

        Y_data= np.zeros((elec_num, lag_number))    

        onsets=all_onsets[-lag_number:-1]
        onsets=np.append(onsets,all_onsets[-1])
    else:

        onsets=[]
        Y_data= np.zeros((elec_num, len(lags)))    

        for i in lags:

            lag_amount = int(i/ 1000 * 512)

            onsets.append(np.minimum(
                t - half_window - 1,
                np.maximum(half_window + 1,
                            np.round(all_onsets[-1]) + lag_amount)))

    
    
    index_onsets=np.asarray(onsets)

   
   
    
    for k in range(np.shape(ecogs)[1]):

        Y1 = np.zeros((len(onsets), 2 * half_window + 1))
        brain_signal=ecogs[:,k]
        
        # subtracting 1 from starts to account for 0-indexing
        starts = index_onsets - half_window - 1
        stops = index_onsets + half_window

        # print(starts)
        # print(stops)
        
        for i, (start, stop) in enumerate(zip(starts, stops)):
            start=int(start)
            stop=int(stop)
            Y1[i, :] = brain_signal[start:stop].reshape(-1)
            
                            
        #if subject==717:
           
        Y_data[k,:] = np.mean(Y1, axis=-1)

    return Y_data

In [11]:
# df=pd.read_csv('/scratch/gpfs/arnab/247_data_read/decoding_df_798_corrected.csv')
df=pd.read_csv('/scratch/gpfs/arnab/247_data_read/decoding_df_798_final.csv')

data=load_pickle('/scratch/gpfs/arnab/247_data_read/last_word_embeddings.pkl')

emb=data['embeddings']
a=data['all_onsets']

df['embeddings']=emb
df['all_onsets']=a

df=df.drop_duplicates(subset=['onsets'])

df=df[df.num_words>min_num_words]
embeddings=df.embeddings.values
all_onsets=df.all_onsets.values

df=df[df.corrupted==0]

In [13]:
N_SAMPLES=30*44100
def pad_or_trim(array, length: int = N_SAMPLES, *, axis: int = -1):
    """
    Pad or trim the audio array to N_SAMPLES, as expected by the encoder.
    """
    if torch.is_tensor(array):
        if array.shape[axis] > length:
            array = array.index_select(
                dim=axis, index=torch.arange(length, device=array.device)
            )

        if array.shape[axis] < length:
            pad_widths = [(0, 0)] * array.ndim
            pad_widths[axis] = (0, length - array.shape[axis])
            array = F.pad(array, [pad for sizes in pad_widths[::-1] for pad in sizes])
    else:
        if array.shape[axis] > length:
            array = array.take(indices=range(length), axis=axis)

        if array.shape[axis] < length:
            pad_widths = [(0, 0)] * array.ndim
            pad_widths[axis] = (0, length - array.shape[axis])
            array = np.pad(array, pad_widths)

    return array

In [26]:
a = torch.randn(4, 4)
print(a)
loss=torch.min(a, -1).values
loss

tensor([[ 0.7095,  0.3646,  0.2364,  1.3628],
        [ 0.4202, -1.0196,  0.7987,  0.3948],
        [-0.7005,  0.2994,  0.8068, -0.0033],
        [-1.2695,  0.3986,  0.5787, -1.4536]])


tensor([ 0.2364, -1.0196, -0.7005, -1.4536])

In [12]:
def all_ecog(elec_id, conv_name,subject ):

    
    elec_id=get_elec_id(subject)
    
    path='/projects/HASSON/247/data/conversations-car/'+str(subject)+'/'
    # conv_name=df[df.conversation_id==conv_id].conversation_name.values[0]
    # q1=glob.glob("*_comp_Y.npy")
    # q1=glob.glob("NY625_*") ## all the conversation
    # # path_elec=path+'/'+q1[0]+'/preprocessed/'
    
    elec_num=len(elec_id)
    
    ecogs=[]
    for k1 in elec_id:
        filename=path+'/'+conv_name+'/preprocessed/'+conv_name+'_electrode_preprocess_file_'+str(k1)+'.mat'
        
        if subject==798:
            filename=path+'/'+conv_name+'/preprocessed_allElec/'+conv_name+'_electrode_preprocess_file_'+str(k1)+'.mat'
        
        e=loadmat(filename)['p1st'].squeeze().astype(np.float32)
        ecogs.append(e)
        
    ecogs = np.asarray(ecogs).T

    return ecogs



In [13]:
subject=798

elec_id=get_elec_id(subject)

conv_names=np.unique(df.conversation_name.values)

if taking_words:
    electrode_data=np.zeros((len(df.conversation_name.values),len(elec_id),lag_number)) 
else:
    electrode_data=np.zeros((len(df.conversation_name.values),len(elec_id),len(lags))) 

embeddings=[]

p=0
for conv_name in conv_names:

    print(p)

    ecogs=all_ecog(elec_id, conv_name,subject)

    df2=df[df.conversation_name==conv_name]

    emb=df2.embeddings.values
    all_onsets=df2.all_onsets.values

    # if p==0:
    #    embeddings=emb
    # else:
    #     embeddings=np.concatenate((embeddings,emb),axis=0)

    for k in range(len(all_onsets)): 

        # print(k)
    
        conv_name=df.conversation_name.values[k]
        x=all_onsets[k]

        if len(x)>= lag_number and x[-1] < len(ecogs[:,0]):
        
            a1= get_elec_data(subject, df, ecogs, conv_name, x, lags, elec_id, lag_number,taking_words)
            electrode_data[p,:,:]=a1
            embeddings.append(emb[k])
            p=p+1

    # p=p+1
electrode_data=electrode_data[:p,:,:]
embeddings=np.asarray(embeddings)
pca = PCA(n_components=emb_dim)
embeddings=pca.fit_transform(embeddings)
embeddings = np.expand_dims(embeddings, axis=1)

0
320
497
889
931
1804
1837
1891
2860
3331
3332
3820
4128
4186


In [14]:
print(np.shape(embeddings))
print(np.shape(electrode_data))


(4394, 1, 50)
(4394, 40, 3)


In [15]:
elec_data=electrode_data[:,electrode,:]
elec_data = np.expand_dims(elec_data, axis=1)
X_train=torch.from_numpy(embeddings[:train_num,:,:])
y_train=torch.from_numpy(elec_data[:train_num,:,:])

X_test=torch.from_numpy(embeddings[train_num:,:,:])
y_test=torch.from_numpy(elec_data[train_num:,:,:])

trainset = torch.utils.data.TensorDataset(X_train, y_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, drop_last=True)

testset = torch.utils.data.TensorDataset(X_test, y_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, drop_last=True)


## Model

In [40]:
class flex_encoding(nn.Module):
    def __init__(self, emb_dim, num_lags):
        super().__init__()
        # the 3 Linear layers of the MLP
        self.fc1 = nn.Linear(emb_dim, 1)
        self.fc2 = nn.Linear(emb_dim, num_lags)
        # self.fc3 = nn.Linear(n_hidden, num_classes)
    
    def forward(self, x):
        
        y_pred= (self.fc1(x))
        y_pred=torch.repeat_interleave(y_pred, 3, dim=-1)
        # v= nn.Softmax(dim=1)(self.fc2(x))
        
    
        return y_pred
        # return y_pred,v

## custom loss function

In [34]:
def bdot(a, b):  ##
    B = a.shape[0]
    S = a.shape[2]
    return torch.bmm(a.view(B, 1, S), b.view(B, S, 1)).reshape(-1)

class CustomLoss(nn.Module):
    def __init__(self):
        super(CustomLoss, self).__init__()

    def forward(self, y_pred, v, targets):
        mse_error = torch.square(targets-y_pred)
        loss=bdot(v, mse_error)
        return loss.mean()  

In [47]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model=flex_encoding(emb_dim, num_lags)
model = model.to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
# loss_fn = CustomLoss()
loss_fn = nn.MSELoss()

In [36]:

print(summary(model, (3, 1, 50)))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1              [-1, 3, 1, 1]              51
Total params: 51
Trainable params: 51
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00
----------------------------------------------------------------
None


In [32]:
x=np.zeros((3,1,1))
x=torch.from_numpy(x)
# x = torch.tensor([1, 2, 3])
x=torch.repeat_interleave(x, 3, dim=-1)
x.size()

torch.Size([3, 1, 3])

In [42]:
for batch_idx, batch in enumerate(trainloader):
        # Every data instance is an input + label pair
        x,y=batch
    
        # Zero your gradients for every batch!
        optimizer.zero_grad()
    
        # Make predictions for this batch
        x = x.to(torch.float32)
        x = x.to(device)
        y = y.to(torch.float32)
        y = y.to(device)
        output1=model(x)

        print(output1.size())

        break

torch.Size([50, 1, 3])


In [45]:
num_lags

10

In [48]:
def train_one_epoch(epoch_index):
    running_loss = 0.
    last_loss = 0.
    
    # Here, we use enumerate(training_loader) instead of
    # iter(training_loader) so that we can track the batch
    # index and do some intra-epoch reporting
    for batch_idx, batch in enumerate(trainloader):
        # Every data instance is an input + label pair
        x,y=batch
    
        # Zero your gradients for every batch!
        optimizer.zero_grad()
    
        # Make predictions for this batch
        x = x.to(torch.float32)
        x = x.to(device)
        y = y.to(torch.float32)
        y = y.to(device)
        [output1,output2]=model(x)
    
        # Compute the loss and its gradients
        loss = loss_fn(output1,output2, y)
        loss.backward()
    
        # Adjust learning weights
        optimizer.step()
    
        # Gather data and report
        running_loss += loss.item()

    last_loss = running_loss / (batch_idx+1) # loss per batch
    print('epoch {} loss: {}'.format(epoch_index, last_loss))
            
    return last_loss

## Training

In [49]:
for epoch in range(EPOCHS):
    print('EPOCH {}:'.format(epoch + 1))

    # Make sure gradient tracking is on, and do a pass over the data
    model.train(True)
    avg_loss = train_one_epoch(epoch)


    running_vloss = 0.0
    # Set the model to evaluation mode, disabling dropout and using population
    # statistics for batch normalization.
    model.eval()

    # Disable gradient computation and reduce memory consumption.
    with torch.no_grad():
        for i, vdata in enumerate(testloader):
            vinputs, vlabels = vdata
            vinputs = vinputs.to(torch.float32)
            vinputs = vinputs.to(device)
            vlabels = vlabels.to(torch.float32)
            vlabels = vlabels.to(device)
            [output1,output2]=model(vinputs)
            voutputs = model(vinputs)
            vloss = loss_fn(output1,output2, vlabels)
            running_vloss += vloss

    avg_vloss = running_vloss / (i + 1)
    print('LOSS train {} valid {}'.format(avg_loss, avg_vloss))

EPOCH 1:


ValueError: too many values to unpack (expected 2)

## draft

In [2]:
a=torch.rand(2,9).chunk(3, dim=-1)
a

(tensor([[0.2207, 0.2205, 0.0548],
         [0.4471, 0.7019, 0.2011]]),
 tensor([[0.3700, 0.6771, 0.2727],
         [0.3288, 0.3360, 0.5386]]),
 tensor([[0.5545, 0.7131, 0.0221],
         [0.6565, 0.6868, 0.2227]]))

In [9]:
x=torch.tensor([1,0,3])

nanned_signal = torch.where(
                        x == 0, torch.tensor(float("nan")), x
                    )
x

tensor([1, 0, 3])

In [10]:
nanned_signal

tensor([1., nan, 3.])

In [3]:
frame_patch_size=1
tube_mask_ratio=0.5
num_patches=1*8*8*40
num_frames=40

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [5]:
tube_mask = (
        torch.zeros(num_patches // (num_frames // frame_patch_size))
        .to(device)
        .to(torch.bool)
    )
tube_mask.shape

torch.Size([64])

In [32]:
signal=torch.rand(2,5, 40,1,8,8)
padding_mask = ~torch.isnan(signal).to(device)

In [30]:
from einops import rearrange

In [33]:
padding_mask=rearrange(padding_mask, 'b c (f pf) (d pd) (h ph) (w pw) -> b f d h w (pd ph pw pf c)',pd=1,
ph=1,
pw=1,
pf=1)
padding_mask = rearrange(padding_mask, "b ... d -> b (...) d")
padding_mask.shape

torch.Size([2, 2560, 5])

In [35]:
padding_mask = torch.all(padding_mask, dim=0)
padding_mask = torch.all(padding_mask, dim=1)

In [37]:
padding_mask.shape

torch.Size([2560])

In [38]:
chn_idx = torch.nonzero(padding_mask[: len(tube_mask)])

In [40]:
mask_idx_candidates = chn_idx[torch.randperm(len(chn_idx))]

In [42]:
tube_idx = mask_idx_candidates[
        : int(
            num_patches
            // (num_frames // frame_patch_size)
            * (1 - tube_mask_ratio)
        )
    ]

In [44]:
tube_mask[tube_idx] = True

In [45]:
tube_mask

tensor([False, False, False,  True, False, False,  True, False,  True, False,
        False,  True,  True,  True,  True, False,  True,  True, False, False,
         True,  True,  True,  True,  True,  True, False, False,  True,  True,
         True, False,  True,  True,  True, False, False, False, False, False,
         True, False,  True, False,  True,  True,  True, False,  True,  True,
        False,  True, False,  True, False,  True, False, False, False, False,
        False, False,  True, False], device='cuda:0')

In [46]:
tube_mask = tube_mask.tile(num_frames // frame_patch_size)

In [53]:
predicted=[]
actual=[]

In [55]:
a=torch.rand(2,3)
print(a)

b=torch.rand(2,3)
print(b)

a1=torch.min(a,dim=-1)
predicted.append((a1.values))

for i in range(b.shape[0]):
    
    actual.append(b[i, a1.indices[i]] )



tensor([[0.0985, 0.1855, 0.4723],
        [0.0242, 0.0191, 0.0627]])
tensor([[0.0772, 0.7507, 0.0346],
        [0.1365, 0.9196, 0.6222]])


In [56]:
predicted

[tensor([0.3031, 0.3336]), tensor([0.0985, 0.0191])]

In [45]:
actual

[tensor(0.0020), tensor(0.6603)]

In [70]:
b1 = [t.cpu().numpy() for t in predicted]

predicted1=np.asarray(b1)

predicted1=predicted1.flatten()
predicted1

array([0.30310047, 0.33358306, 0.09848493, 0.01910239], dtype=float32)

In [71]:
actual

[tensor(0.6246), tensor(0.6726), tensor(0.0772), tensor(0.9196)]